In [81]:
from string import ascii_letters

import pandas as pd

from utils.paths import BOOKS_PATH, AUTHORS_PATH

In [82]:
books_df = pd.read_csv(BOOKS_PATH)
authors_df = pd.read_csv(AUTHORS_PATH)

/tmp/ipykernel_102037/587830805.py:1: DtypeWarning: Columns (66,67,68,69,70,71,72,73,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv(BOOKS_PATH)


In [83]:
books_df['author_href']

0           https://lubimyczytac.pl/autor/3701/j-k-rowling
1           https://lubimyczytac.pl/autor/3701/j-k-rowling
2           https://lubimyczytac.pl/autor/3701/j-k-rowling
3           https://lubimyczytac.pl/autor/3701/j-k-rowling
4        https://lubimyczytac.pl/autor/14276/antoine-de...
                               ...                        
10015      https://lubimyczytac.pl/autor/230535/sally-page
10016    https://lubimyczytac.pl/autor/115156/natalia-s...
10017    https://lubimyczytac.pl/autor/20127/harry-harr...
10018    https://lubimyczytac.pl/autor/153763/anna-ciar...
10019    https://lubimyczytac.pl/autor/31358/arnaldur-i...
Name: author_href, Length: 10020, dtype: object

In [84]:
authors_df['author_url']

0          https://lubimyczytac.pl/autor/3701/j-k-rowling
1       https://lubimyczytac.pl/autor/14276/antoine-de...
2        https://lubimyczytac.pl/autor/3216/j-r-r-tolkien
3       https://lubimyczytac.pl/autor/10474/george-orwell
4       https://lubimyczytac.pl/autor/22701/michail-bu...
                              ...                        
3203    https://lubimyczytac.pl/autor/71329/david-r-ha...
3204    https://lubimyczytac.pl/autor/41653/marek-racz...
3205    https://lubimyczytac.pl/autor/30146/william-irwin
3206    https://lubimyczytac.pl/autor/33931/thomas-pyn...
3207      https://lubimyczytac.pl/autor/230535/sally-page
Name: author_url, Length: 3208, dtype: object

In [85]:
data_df = books_df.join(authors_df.set_index('author_url'), on='author_href', how='inner')

In [86]:
# check if cycles always end with <cycle name> (<string> <number>)
data_df['Cykl'].dropna().str.extract(r'(\d)+').fillna(0).astype(int)[0].unique()

array([1, 2, 3, 4, 5, 6, 7, 0, 8, 9])

In [87]:
data_df['part_of_cycle'] = data_df['Cykl'].str.extract(r'(\d)+').fillna(0).astype(int)[0]

In [88]:
data_df['part_of_cycle'].sort_values(ascending=False)[:3]

9917    9
1982    9
7678    9
Name: part_of_cycle, dtype: int64

In [89]:
data_df.loc[data_df['part_of_cycle'].sort_values(ascending=False)[:3].index, :]

,author,author_href,description,number_of_user_opinions,number_of_user_ratings,number_of_discussions,Kategoria,Format,Cykl,Tytuł oryginału,...,author_href18,author_name,author_average_rating,author_number_of_people_read,author_number_of_people_wants_to_read,author_date_of_birth,author_number_of_fans,author_number_of_books_written,author_number_of_awards,part_of_cycle
9917,Robert Muchamore,https://lubimyczytac.pl/autor/6291/robert-much...,9 tom popularnej serii Cherub opowiadającej o...,22,555,0,"kryminał, sensacja, thriller",papier,CHERUB (tom 9),The Sleepwalker,...,NaN,Robert Muchamore,7.6,2361,4420,25.12.1972,221,34,0,9
1982,Agatha Christie,https://lubimyczytac.pl/autor/14083/agatha-chr...,"Panna Marple, dzięki hojności swego siostrzeń...",172,2445,0,"kryminał, sensacja, thriller",papier,Panna Marple (tom 9),A Caribbean mistery,...,NaN,Agatha Christie,7.0,58293,132793,15.09.1890,6416,186,0,9
7678,Katarzyna Bonda,https://lubimyczytac.pl/autor/14572/katarzyna-...,"Co sprawiło, że w chorym umyśle zrodził się p...",165,741,0,"kryminał, sensacja, thriller",papier,Hubert Meyer (tom 9),NaN,...,NaN,Katarzyna Bonda,6.9,31880,39022,przybliżona),1908,43,0,9


In [90]:
# drop records without number of pages
data_df = data_df.dropna(subset=['Liczba stron'])

In [91]:
numeric_columns = data_df.select_dtypes(include='number').columns

In [92]:
float_columns = 'author_average_rating'
integer_columns = numeric_columns.drop(float_columns)

In [93]:
# integer cols where there is nans
data_df.loc[:, integer_columns] = data_df.loc[:, integer_columns].fillna(0).astype(int)

In [94]:
data_df[integer_columns]

,number_of_user_opinions,number_of_user_ratings,number_of_discussions,Liczba stron,number_of_people_read,number_of_people_has,number_of_people_favorite,number_of_people_wants_to_read,number_of_people_wants_as_gift,number_of_people_currently_read,...,rating_4,rating_3,rating_2,rating_1,author_number_of_people_read,author_number_of_people_wants_to_read,author_number_of_fans,author_number_of_books_written,author_number_of_awards,part_of_cycle
0,5156,85009,1,320.0,124692,30249,11410.0,6210,990.0,849.0,...,501,490,67,145,130174,75668,11479,40,1,1
1,2305,70818,0,356.0,105466,25003,8972.0,4535,883.0,501.0,...,345,404,26,67,130174,75668,11479,40,1,2
2,2241,67918,0,456.0,101297,23587,10242.0,5041,938.0,463.0,...,200,235,20,46,130174,75668,11479,40,1,3
3,1892,64439,0,764.0,96528,22705,9109.0,5211,971.0,632.0,...,244,274,27,55,130174,75668,11479,40,1,4
4,4264,63437,0,144.0,91987,16084,7102.0,3781,536.0,312.0,...,1040,1139,335,428,87296,8410,681,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10015,154,573,1,352.0,637,124,9.0,558,0.0,17.0,...,16,10,3,3,638,559,5,2,0,0
10016,138,573,0,352.0,711,119,19.0,282,0.0,0.0,...,11,7,2,0,6924,12208,370,42,0,5
10017,90,546,0,288.0,708,273,18.0,682,0.0,10.0,...,15,10,0,3,3431,6316,133,99,0,0
10018,94,558,0,302.0,719,232,26.0,686,11.0,18.0,...,34,31,20,13,4313,4458,58,5,0,0


# imputacja tytulow

In [95]:
data_df.columns

Index(['author', 'author_href', 'description', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions', 'Kategoria',
       'Format', 'Cykl', 'Tytuł oryginału', 'Wydawnictwo', 'Data wydania',
       'Data 1. wyd. pol.', 'Data 1. wydania', 'Liczba stron', 'Czas czytania',
       'Język', 'ISBN', 'Tłumacz', 'Ekranizacje', 'number_of_people_read',
       'number_of_people_has', 'number_of_people_favorite',
       'number_of_people_wants_to_read', 'number_of_people_wants_as_gift',
       'number_of_people_currently_read', 'tags', 'rating_10', 'rating_9',
       'rating_8', 'rating_7', 'rating_6', 'rating_5', 'rating_4', 'rating_3',
       'rating_2', 'rating_1', 'url', 'Inne', 'author1', 'author_href1',
       'Seria', 'author2', 'author_href2', 'author3', 'author_href3',
       'author4', 'author_href4', 'author5', 'author_href5', 'author6',
       'author_href6', 'author7', 'author_href7', 'author8', 'author_href8',
       'author9', 'author_href9', 'author10'

In [96]:
missing_title = data_df['Tytuł oryginału'].isna()

In [97]:
data_df.loc[missing_title, 'Tytuł oryginału'] = data_df.loc[missing_title, 'url'].str.split('/').str[-1].str.split('-').str.join(' ').str.title()

In [98]:
data_df['part_of_cycle']

0        1
1        2
2        3
3        4
4        0
        ..
10015    0
10016    5
10017    0
10018    0
10019    8
Name: part_of_cycle, Length: 9738, dtype: int64

# cleaning

In [99]:
data_df.columns

Index(['author', 'author_href', 'description', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions', 'Kategoria',
       'Format', 'Cykl', 'Tytuł oryginału', 'Wydawnictwo', 'Data wydania',
       'Data 1. wyd. pol.', 'Data 1. wydania', 'Liczba stron', 'Czas czytania',
       'Język', 'ISBN', 'Tłumacz', 'Ekranizacje', 'number_of_people_read',
       'number_of_people_has', 'number_of_people_favorite',
       'number_of_people_wants_to_read', 'number_of_people_wants_as_gift',
       'number_of_people_currently_read', 'tags', 'rating_10', 'rating_9',
       'rating_8', 'rating_7', 'rating_6', 'rating_5', 'rating_4', 'rating_3',
       'rating_2', 'rating_1', 'url', 'Inne', 'author1', 'author_href1',
       'Seria', 'author2', 'author_href2', 'author3', 'author_href3',
       'author4', 'author_href4', 'author5', 'author_href5', 'author6',
       'author_href6', 'author7', 'author_href7', 'author8', 'author_href8',
       'author9', 'author_href9', 'author10'

## author quantitative features:
author_average_rating
author_number_of_people_read
author_number_of_people_wants_to_read
author_date_of_birth ?
author_number_of_fans
author_number_of_books_written
author_number_of_awards
## book quantitative features
description_length (engineered)
number_of_user_opinions
number_of_user_ratings
number_of_discussions
number_of_people_read
Data wydania
Liczba stron
## book qualitative features
Kategoria
part_of_cycle (engineered)
Język
number_of_people_has
number_of_people_favorite
number_of_people_wants_to_read
number_of_people_wants_as_gift
number_of_people_currently_read

In [100]:
author_quant_features = [
    "author_average_rating",
    "author_number_of_people_read",
    "author_number_of_people_wants_to_read",
    "author_date_of_birth",  # Możliwe, że wymaga dodatkowego przetworzenia
    "author_number_of_fans",
    "author_number_of_books_written",
    "author_number_of_awards"
]

# Cechy ilościowe związane z książką
book_quant_features = [
    "number_of_user_opinions",
    "number_of_user_ratings",
    "number_of_discussions",
    "number_of_people_read",
    "Data wydania",  # to be transformed
    "Liczba stron",
    "Cykl"
] + [f'rating_{i}' for i in range(1, 11)]

# Cechy jakościowe związane z książką
book_qual_features = [
    "Kategoria",
    "part_of_cycle",  # engineered
    "Język",
    "number_of_people_has",
    "number_of_people_favorite",
    "number_of_people_wants_to_read",
    "number_of_people_wants_as_gift",
    "number_of_people_currently_read"
]

In [101]:
all_features = author_quant_features + book_quant_features + book_qual_features

In [102]:
data_df[all_features].columns

Index(['author_average_rating', 'author_number_of_people_read',
       'author_number_of_people_wants_to_read', 'author_date_of_birth',
       'author_number_of_fans', 'author_number_of_books_written',
       'author_number_of_awards', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions',
       'number_of_people_read', 'Data wydania', 'Liczba stron', 'Cykl',
       'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
       'rating_7', 'rating_8', 'rating_9', 'rating_10', 'Kategoria',
       'part_of_cycle', 'Język', 'number_of_people_has',
       'number_of_people_favorite', 'number_of_people_wants_to_read',
       'number_of_people_wants_as_gift', 'number_of_people_currently_read'],
      dtype='object')

In [103]:
data_df.loc[:, 'author_date_of_birth'] = pd.to_datetime(data_df['author_date_of_birth'], format='mixed', errors='coerce')

In [104]:
data_df.loc[:, 'author_date_of_birth']

0        1965-07-31 00:00:00
1        1965-07-31 00:00:00
2        1965-07-31 00:00:00
3        1965-07-31 00:00:00
4        1900-06-29 00:00:00
                ...         
10015    1937-08-05 00:00:00
10016    1884-01-02 00:00:00
10017    1925-12-03 00:00:00
10018    1984-01-05 00:00:00
10019    1961-01-28 00:00:00
Name: author_date_of_birth, Length: 9738, dtype: object

In [105]:
data_df[all_features].columns

Index(['author_average_rating', 'author_number_of_people_read',
       'author_number_of_people_wants_to_read', 'author_date_of_birth',
       'author_number_of_fans', 'author_number_of_books_written',
       'author_number_of_awards', 'number_of_user_opinions',
       'number_of_user_ratings', 'number_of_discussions',
       'number_of_people_read', 'Data wydania', 'Liczba stron', 'Cykl',
       'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6',
       'rating_7', 'rating_8', 'rating_9', 'rating_10', 'Kategoria',
       'part_of_cycle', 'Język', 'number_of_people_has',
       'number_of_people_favorite', 'number_of_people_wants_to_read',
       'number_of_people_wants_as_gift', 'number_of_people_currently_read'],
      dtype='object')

In [106]:
rename_dict = {
    "Data wydania": "publication_date",
    "Liczba stron": "number_of_pages",
    "Cykl": "series",
    "Kategoria": "category",
    "Język": "language"
}

In [107]:
renamed_cols = [rename_dict.get(col, col) for col in data_df[all_features].columns]

In [109]:
renamed_cols

['author_average_rating',
 'author_number_of_people_read',
 'author_number_of_people_wants_to_read',
 'author_date_of_birth',
 'author_number_of_fans',
 'author_number_of_books_written',
 'author_number_of_awards',
 'number_of_user_opinions',
 'number_of_user_ratings',
 'number_of_discussions',
 'number_of_people_read',
 'publication_date',
 'number_of_pages',
 'series',
 'rating_1',
 'rating_2',
 'rating_3',
 'rating_4',
 'rating_5',
 'rating_6',
 'rating_7',
 'rating_8',
 'rating_9',
 'rating_10',
 'category',
 'part_of_cycle',
 'language',
 'number_of_people_has',
 'number_of_people_favorite',
 'number_of_people_wants_to_read',
 'number_of_people_wants_as_gift',
 'number_of_people_currently_read']

In [110]:
# data_df = data_df.rename(columns=rename_dict)
data_df = data_df.rename(columns=rename_dict)

In [115]:
data_df[renamed_cols].to_csv('data/cleaned_data.csv', index=False)